In [1]:
import import_ipynb
from mydata import mydata
import torch
import torch.nn as nn
from torch import optim
from transformers import *
import pandas as pd
import ast
import copy
import os
from time import strftime,gmtime
from opencc import OpenCC
import pyprind



importing Jupyter notebook from mydata.ipynb
Train:4520 Test:503


In [ ]:
"""
# Model
class bertwwm(nn.Module):
    def __init__(self):
        super(bertwwm,self).__init__()
        self.bert_model = BertModel.from_pretrained("hfl/chinese-bert-wwm")
        #self.tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm")
        self.bi_decoder = nn.Sequential(
            nn.Linear(768,1)
            ,nn.Dropout(0.1)
            ,nn.Sigmoid()
        )
        self.start_Linear = nn.Sequential(
            nn.Linear(768,512)
            ,nn.Dropout(0.1)
            ,nn.Sigmoid() 
        )
        self.end_Linear = nn.Sequential(
            nn.Linear(768,512)
            ,nn.Dropout(0.1)
            ,nn.Sigmoid() 
        )
    def forward(self,input_ids):
        #print(input_ids.shape)
        # input a string
        all_hidden_states, all_attentions = self.bert_model(input_ids)[-2:]
        binary = self.bi_decoder(all_attentions).reshape(1)
        s = self.start_Linear(all_attentions).reshape(512)
        e = self.end_Linear(all_attentions).reshape(512)
       # print(binary.shape,binary)
        return binary,s,e
"""

In [2]:
# Model
class bertwwm_pos(nn.Module):
    def __init__(self):
        super(bertwwm_pos,self).__init__()
        self.bert_model = BertModel.from_pretrained("hfl/chinese-bert-wwm")
        #self.tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm")
        self.start_Linear = nn.Sequential(
            nn.Linear(768,512)
            ,nn.Dropout(0.1)
            ,nn.Sigmoid() 
        )
        self.end_Linear = nn.Sequential(
            nn.Linear(768,512)
            ,nn.Dropout(0.1)
            ,nn.Sigmoid() 
        )
    def forward(self,input_ids):
        #print(input_ids.shape)
        # input a string
        all_hidden_states, all_attentions = self.bert_model(input_ids)[-2:]
        #binary = self.bi_decoder(all_attentions).reshape(1)
        s = self.start_Linear(all_attentions).reshape(512)
        e = self.end_Linear(all_attentions).reshape(512)
       # print(binary.shape,binary)
        return s,e

In [3]:
"""
def test(model,data,device):
    print('Testing...')
    size,loss,acc,not_acc = 0,0,0,0
    test_num = len(data.test)
    criterion = nn.BCELoss()
    tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm")
    model.eval()
    print(f'Testing data: {test_num}')
    with torch.no_grad():
        for content,name,s,e in data.test:  
            if len(name) is 0:    
                label = torch.Tensor([0]).to('cpu')
            else:
                label = torch.Tensor([1]).to('cpu')
            size +=1 
            input_ids = torch.tensor([tokenizer.encode(str(content),max_length=512,truncation=True)]).to(device)
            b_pred,s_pred,e_pred = model(input_ids).to('cpu')
            a_loss = criterion(pred,label) 
            loss += a_loss
            if len(name) is 0 and pred.item() < 0.4:
                #print(pred.item())
                acc +=1
            elif len(name) is not 0 and pred.item() >0.4:
                acc +=1    
    loss /= test_num
    return loss,acc
"""

'\ndef test(model,data,device):\n    print(\'Testing...\')\n    size,loss,acc,not_acc = 0,0,0,0\n    test_num = len(data.test)\n    criterion = nn.BCELoss()\n    tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm")\n    model.eval()\n    print(f\'Testing data: {test_num}\')\n    with torch.no_grad():\n        for content,name,s,e in data.test:  \n            if len(name) is 0:    \n                label = torch.Tensor([0]).to(\'cpu\')\n            else:\n                label = torch.Tensor([1]).to(\'cpu\')\n            size +=1 \n            input_ids = torch.tensor([tokenizer.encode(str(content),max_length=512,truncation=True)]).to(device)\n            b_pred,s_pred,e_pred = model(input_ids).to(\'cpu\')\n            a_loss = criterion(pred,label) \n            loss += a_loss\n            if len(name) is 0 and pred.item() < 0.4:\n                #print(pred.item())\n                acc +=1\n            elif len(name) is not 0 and pred.item() >0.4:\n                acc +=1

In [4]:
def test(model,data,device):
    print('Testing...')
    size,loss,acc,not_acc = 0,0,0,0
    test_num = len(data.test)
    criterion = nn.BCELoss()
    cross = nn.CrossEntropyLoss()
    tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm")
    model.eval()
    print(f'Testing data: {test_num}')
    B_loss,S_loss,E_loss = 0,0,0
    with torch.no_grad():
        for content,name,s,e in data.train:
            i +=1 
            input_ids = torch.tensor([tokenizer.encode(str(content),max_length=512,truncation=True)]).to(device)   
            b_pred,s_pred,e_pred = model(input_ids)
            b_pred = b_pred.to('cpu')
            s_pred = s_pred.to('cpu')
            e_pred = e_pred.to('cpu')
        # Binary:
            if len(name) is 0:    
                b_label = torch.Tensor([0]).to('cpu')
            else:
                b_label = torch.Tensor([1]).to('cpu')
            
            if len(name) is 0 and pred.item() < 0.4:
                acc +=1
            elif len(name) is not 0 and pred.item() >0.4:
                acc +=1 
        # Start
            s_label = [0]*512
            for s_ in s:
                s_label[s_] = 1
            s_label = torch.Tensor(s_label).to('cpu')
            print(s_label)
        # End
            e_label = [0]*512
            for e_ in e:
                e_label[e_] = 1
            e_label = torch.Tensor(e_label).to('cpu')
        # count loss
            a_b_loss = criterion(b_pred,b_label)
            a_s_loss = criterion(s_pred,s_label)
            a_e_loss = criterion(e_pred,e_label)
            B_loss += a_b_loss
            S_loss += a_s_loss
            E_loss += a_e_loss
            
    # Averge        
    B_loss /= test_num
    S_loss /= test_num
    E_loss /= test_num
    
    return B_loss,S_loss,E_loss,acc

In [5]:
# lr  : learning rate
# w_d :weight_decay

def train(data,lr_rate ,w_d,device):
    model = bertwwm_pos().to(device)
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = optim.Adadelta(parameters, lr=lr_rate, weight_decay=w_d)
    criterion = nn.BCELoss()
    cross = nn.CrossEntropyLoss()
    #cc = OpenCC('tw2sp')
    max_test_acc = 0
    tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm")
    i = 0
    model.train()
    optimizer.zero_grad()
    loss = 0
    pbar = pyprind.ProgBar(len(data.train))
    for content,name,s,e in data.train:
        i +=1 
        
        input_ids = torch.tensor([tokenizer.encode(str(content),max_length=512,truncation=True)]).to(device)   
        
        #b_pred,s_pred,e_pred = model(input_ids)
        
        #b_pred = b_pred.to('cpu')
        
        s_pred,e_pred = model(input_ids)
        del input_ids
        
        s_pred = s_pred.to('cpu')
        e_pred = e_pred.to('cpu')
               
        for task in range(0,3):
            # Binary:
            if task is -1:
                if len(name) is 0:    
                    b_label = torch.Tensor([0]).to('cpu')
                else:
                    b_label = torch.Tensor([1]).to('cpu')
                a_b_loss = criterion(b_pred,b_label)
                a_b_loss.backward(retain_graph=True)
                optimizer.step()
                optimizer.zero_grad()
             # Start
            elif task is 1:
                s_label = [0]*512
                for s_ in s:
                    s_label[s_] = 1
                s_label = torch.Tensor(s_label).to('cpu')
                a_s_loss = criterion(s_pred,s_label)
                a_s_loss.backward(retain_graph=True)
                optimizer.step()
                optimizer.zero_grad()
            elif task is -1:
            # End
                e_label = [0]*512
                for e_ in e:
                    e_label[e_] = 1
                e_label = torch.Tensor(e_label).to('cpu')
                a_e_loss = criterion(e_pred,e_label)
                a_e_loss.backward(retain_graph=True)
                optimizer.step()
                optimizer.zero_grad()
            #optimizer.step()
            
         # count loss 
        # backward    
        #loss += (a_b_loss + a_s_loss + a_e_loss)/3
        #print(f'Train loss {loss}') 
        #if (i+1) % 4 is 0:
        
            #print(f'Batch loss is {loss/8}')
            #loss = 0
        pbar.update()
        """
        if (i+1) % 1000 is 0:
            test_loss,test_acc = test(model,data,device)
            print(f'Test Loss is {test_loss} ACC is {test_acc}')
            if test_acc > max_test_acc:
                print('Better model!')
                best_model = copy.deepcopy(model.state_dict())
        model.to(device)
        model.train()
        """
    best_model = copy.deepcopy(model.state_dict())
    return best_model

In [6]:

    
device = torch.device('cuda:0')
data = mydata('./train_fix.csv')
print('Data load down')
mode = 'train'



Train:4520 Test:503
Data load down


In [7]:
if mode is 'train':
    best_model = train(data,0.005,5e-5,device)
    
    if not os.path.exists('saved_models'):
        os.makedirs('saved_models')    
    modeltime = strftime('%H_%M_%S', gmtime()) 
    modelname = 'bertWWM_SE_'+ modeltime
    torch.save(best_model, f'saved_models/{modelname}.pt')
    print(f'Train end, model name is {modelname}.pt')
    test_model = bertwwm().to(device)
    test_model.load_state_dict(torch.load(f'saved_models/{modelname}.pt'))
    test_b_loss,test_s_loss,test_e_loss,test_acc = test(test_model,data,device)
    print(f'LOSS: B{test_b_loss} S{test_s_loss} E{test_e_loss} ACC is {test_acc}')
    
elif mode is 'testb':
    modelname = 'bertWWM_SE_15_37_51.pt'
    test_model = bertwwm().to(device)
    test_model.load_state_dict(torch.load(f'saved_models/{modelname}'))
    test_b_loss,test_s_loss,test_e_loss,test_acc = test(test_model,data,device)
    print(f'LOSS: B{test_b_loss} S{test_s_loss} E{test_e_loss} ACC is {test_acc}')

elif mode is 'tests':
    modelname = 'bertWWM_SE_15_37_51.pt'
    test_model = bertwwm_pos().to(device)
    test_model.load_state_dict(torch.load(f'saved_models/{modelname}'))
    test_b_loss,test_s_loss,test_e_loss,test_acc = test(test_model,data,device)
    print(f'LOSS: B{test_b_loss} S{test_s_loss} E{test_e_loss} ACC is {test_acc}')



0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:04:47


Train end, model name is bertWWM_SE_15_37_51.pt


RuntimeError: Error(s) in loading state_dict for bertwwm:
	Missing key(s) in state_dict: "bi_decoder.0.weight", "bi_decoder.0.bias". 

In [ ]:
#if __name__ == '__main__':
#    main()

In [ ]:
# bertWWM_06_41_57.pt : 500/503
# bertWWM_06_52_12.pt.pt : 500/503
# bertWWM_09_01_54.pt : 501/503

In [ ]:
"""
cross = nn.BCELoss()
x = torch.Tensor([0.3,0.2,0.1])
y = torch.Tensor([1,0,0])
print(x.shape,y.shape)
z = cross(x,y)
z
"""